In [15]:
#this is to digitize lithologies for better processing. It will be easier to plot them
def str_array2floats(strarray):
    floats=[]
    for s in strarray:
        try:
            floats.append(float(s))
        except:
            floats.append(np.nan)
    return floats


# np.array(lith_labels).shape
def merge_similar_layers(loc_layer,loc_lith_labels):
    res_layer=[]
    res_label=[]
    for a in np.unique(loc_lith_labels):
        single_layer_prop=0
        if a!=-1:
            single_layer_prop=0
            for n,l in zip(loc_lith_labels,loc_layer):
                if a==n:
                    single_layer_prop += l
            res_layer.append(single_layer_prop)
            res_label.append(a)
    return res_layer,res_label

# for processing purpose at all locations, same number of layers and labels prepared. 
def make_same_numb_layers(master_l_thicknesses,lith_labels):
    layersinlocations=[len(l) for l in master_l_thicknesses]
    maxls=max(layersinlocations)
    j=-1
    for ml in master_l_thicknesses:
        j +=1
        if len(ml)<maxls:
            for i in range(maxls-len(ml)):
                ml.append(0)
                lith_labels[j]=np.append(lith_labels[j],-1)
    loc_layers=np.array(master_l_thicknesses)
    return loc_layers,np.array(lith_labels)

#it is thought that if all the locations have layer thickness information according to sorted layers, it will be lot easier to 
#plot or process the information so the function for sorting single location and creating zero thickness layer for missing 
#layer
def get_sortedlayers4loc(mod_loc_layers,mod_loc_layer_lbls,unique_lbls):
    sorted_loc_layer=[]
    for ul in unique_lbls:
        sorted_loc_layer.append(0)
        for e,l in zip(mod_loc_layers,mod_loc_layer_lbls):
            if l==ul:
                sorted_loc_layer[-1]=e
#     sorted_loc_layer[-1]
    return sorted_loc_layer
def get_depths(thickness_layer_array,elevation):
    depths=[-elevation]
    d=-elevation   
    for e in thickness_layer_array:
        d += e
        depths.append(d)
    return depths

In [16]:
# Modules and data loading
import numpy as np
import pandas as pd
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import re
import math
from scipy.interpolate import griddata,interp2d
# import mpl_toolkits.mplot3d.art3d.Poly3DCollection
from   mpl_toolkits.mplot3d.art3d import Poly3DCollection
sys.path.append('..')
from pickle_fns import *
base_folder=r'D:\Ameyem Office\Projects\Electric surveys\Easwar files\jalaun\\'
base_folder=r'D:\Ameyem Office\Projects\Electric surveys\Easwar files\Mahoba\\'
vesdf,data_dfs=load_pkl(base_folder+'vesdf_datadf.pkl')
# vesdf,data_dfs=load_pkl('tikamgarh.pkl')
E,N,Ele=vesdf.Easting.values.astype(np.float),vesdf.Northing.values.astype(np.float),str_array2floats(vesdf.RL.values)
# Ele=[float(re.findall('\d*\.\d+|\d+',E)[0]) for E in Ele]
# E,N,data_dfs[0]

nlayers=max([len(d) for d in data_dfs])
all_liths=[[]]*nlayers
all_liths=[]
for i in range(nlayers):
    all_liths.append([])
for df in data_dfs:
    for i in range(len(df['Interpreted Lithology'])):

        all_liths[i].append(df['Interpreted Lithology'].iloc[i])
#         break
#     break
for a in all_liths :
    print(np.unique(a))

['Dry alluvium soil' 'Dry soil with kankar' 'Soil with Kankar'
 'Soil with kankar' 'Soil with kankars' 'Yellowish sandy clay']
['Alluvium soil' 'Clay and Weathered pink granite may be under saturation'
 'Clay with kankar' 'Clay with quartz and weathered pink granite'
 'Course sand with weathered granite'
 'Fractured pink Granite, may be under saturation' 'Soil with Kankar'
 'Soil with kankar' 'Weathered grey granite'
 'Weathered pink Granite to pink granite mafic mineral increase'
 'Weathered pink granite' 'Yellowish sandy clay'
 'quartz and weathered pink granite Clay']
['Clay and Weathered pink granite may be under saturation'
 'Clay with kankar' 'Clay with quartz and weathered pink granite'
 'Granite mafic mineral increase-May be under Saturation'
 'Grey granite mafic mineral increase' 'Hard massive granite'
 'Pink granite mafic mineral increase may be fractured and under saturation'
 'Pink granite slightly black&white  fine to meadium grains-May be under Saturation'
 'Soil with Kan

In [17]:
#mohaba after editing

lith_dict={
    0:['Dry alluvium soil', 'Dry soil with kankar', 'Soil with Kankar', 
 'Soil with kankar', 'Soil with kankars', 'Yellowish sandy clay'],
#     1:['quartz and weathered pink granite Clay','Alluvium soil',

#  'Clay with kankar',  
        
        
#     ],
    1:['quartz and weathered pink granite Clay','Alluvium soil','Clay with kankar',
        
        'Weathered grey granite', 'Weathered pink granite','Weathered pink  granite',
         'Course sand with weathered granite','Clay and Weathered pink granite may be under saturation','Clay with quartz and weathered pink granite',
    ],
    2:['Pink granite mafic mineral increase may be fractured and under saturation','Fractured pink Granite, may be under saturation',
        'Weathered pink Granite to pink granite mafic mineral increase','Granite mafic mineral increase-May be under Saturation',
       'Grey granite mafic mineral increase','Pink granite slightly black&white  fine to meadium grains-May be under Saturation',
    'Hard massive granite',],
#     4:['Hard massive granite',]
}


# lith_dict={
#     0:['Dry alluvium soil', 'Dry soil with kankar', 'Soil with Kankar',
#  'Soil with kankar', 'Soil with kankars', 'Yellowish sandy clay'],
#     1:[
#         'Alluvium soil','Clay with quartz and weathered pink granite','Clay and Weathered pink granite may be under saturation','Weathered pink  granite',
#  'Clay with kankar',  
        
        
#     ],
#     2:[
#          'Weathered grey granite', 'Weathered pink granite','Course sand with weathered granite','Fractured pink Granite, may be under saturation',
#     ],
#     3:[
#         'Weathered pink Granite to pink granite mafic mineral increase','Granite mafic mineral increase-May be under Saturation',
#        'Grey granite mafic mineral increase','Pink granite slightly black&white  fine to meadium grains-May be under Saturation',
#     ],
#     4:['Hard massive granite',]
# }

lith_dict_summay={
    0:'Soil/Dry soil/ Soil with kankars',
#     1:'Clayee/Weathered granite/Fractured granite',
    1:'Weathered granite',
#     3:'Weathered Granite to granite mafic mineral increase',
    2:'Hard massive granite'
}

save_pkl(base_folder+'lith_dict.pkl',lith_dict)

lith=[]
for df in data_dfs:
#     print(df.iloc[0])
    lith.extend(df['Interpreted Lithology'].values)
# data_dfs[0]
np.unique(np.array(lith))

array(['Alluvium soil',
       'Clay and Weathered pink granite may be under saturation',
       'Clay with kankar', 'Clay with quartz and weathered pink granite',
       'Course sand with weathered granite', 'Dry alluvium soil',
       'Dry soil with kankar',
       'Fractured pink Granite, may be under saturation',
       'Granite mafic mineral increase-May be under Saturation',
       'Grey granite mafic mineral increase', 'Hard massive granite',
       'Pink granite mafic mineral increase may be fractured and under saturation',
       'Pink granite slightly black&white  fine to meadium grains-May be under Saturation',
       'Soil with Kankar', 'Soil with kankar', 'Soil with kankars',
       'Weathered grey granite', 'Weathered pink  granite',
       'Weathered pink Granite to pink granite mafic mineral increase',
       'Weathered pink granite', 'Yellowish sandy clay',
       'quartz and weathered pink granite Clay'], 
      dtype='<U81')

In [18]:
uall_liths=[np.unique(a) for a in all_liths]
[len(a) for a in all_liths],[len(a) for a in uall_liths],uall_liths
ualiths=[]
for u in uall_liths:
    ualiths.extend(u)
len(ualiths ),len(np.unique(ualiths) ),np.unique(ualiths) 

def find_not_in_lithDict(all_unique_liths,lith_dict):
    all_lith=[]
    for j in lith_dict:
        all_lith.extend(lith_dict[j] )
    to_be_included=[]
    for l in all_unique_liths:
        if not l in all_lith:
            to_be_included.append(l)
    return to_be_included
find_not_in_lithDict(np.unique(ualiths),lith_dict) 

[]

In [19]:
def get_lables(df,lith_dict):
    labels=[]
    for s in df['Interpreted Lithology'].values:
        print(s)
        for i in range(len(lith_dict)):
            if s in lith_dict[i]:
#                 print(lith_dict[i])
                if i in labels and i!=labels[-1]:
                    labels.append(i*10)
                else:
                    labels.append(i)
                break
    return labels
get_lables(df,lith_dict),df
#utilization of get labels
for df in data_dfs:
    print('a')
    print(get_lables(df,lith_dict) )
    df['lith_label']=get_lables(df,lith_dict)    
lith_labels=np.array(df['lith_label'].values)

Soil with Kankar
Clay and Weathered pink granite may be under saturation
Weathered pink Granite to pink granite mafic mineral increase
a
Dry soil with kankar
Clay with kankar
Clay with kankar
Clay with kankar
Hard massive granite
[0, 1, 1, 1, 2]
Dry soil with kankar
Clay with kankar
Clay with kankar
Clay with kankar
Hard massive granite
a
Soil with Kankar
Clay with quartz and weathered pink granite
Pink granite slightly black&white  fine to meadium grains-May be under Saturation
[0, 1, 2]
Soil with Kankar
Clay with quartz and weathered pink granite
Pink granite slightly black&white  fine to meadium grains-May be under Saturation
a
Yellowish sandy clay
Yellowish sandy clay
Weathered pink granite
Weathered pink Granite to pink granite mafic mineral increase
[0, 0, 1, 2]
Yellowish sandy clay
Yellowish sandy clay
Weathered pink granite
Weathered pink Granite to pink granite mafic mineral increase
a
Yellowish sandy clay
quartz and weathered pink granite Clay
Weathered pink Granite to pink g

In [20]:
for vv,df in zip(vesdf['VES No.'].values,data_dfs):
#     print(vv)
    if vv=='121':
        print(df)

  Layer No. Resistivity(Ωm) Thickness(m) Depth (m)   Elev.  \
0         1          92.977      0.98922   0.98922  152.01   
1         2          10.064       10.184    11.173  141.83   
2         3          425.14           --        --      --   

                               Interpreted Lithology  lith_label  
0                                  Dry alluvium soil           0  
1                                      Alluvium soil           1  
2  Weathered pink Granite to pink granite mafic m...           2  


In [21]:
# lith_labels

#construction of lith labels for layers or thicknesses

np.set_printoptions(precision=3, suppress=True)
master_l_thicknesses=[]
lith_labels=[]
liths=[]
max_strata_thickness=400
for df in data_dfs:
#     print(df['Thickness(m)'].values.astype(np.float))
    layers_values=str_array2floats(df['Thickness(m)'].values)
    lastlayer_thickness=[max_strata_thickness-np.nansum(layers_values) if math.isnan(x) else x for x in layers_values]
#     print(lastlayer_thickness)
    master_l_thicknesses.append(lastlayer_thickness)
    lith_labels.append(np.array(df['lith_label'].values))
    liths.append(np.array(df['Interpreted Lithology'].values))
elevations=str_array2floats(vesdf.RL.values)
# layers_values,np.nansum(layers_values),master_l_thicknesses,lith_labels
# with lith dict here most of the layers are repeated.
# And similar layers need to be merged
for mt,ll in zip(master_l_thicknesses,lith_labels):
    print(mt,'\t\t\t',ll)

[0.80102, 0.38112, 0.62656, 0.26081, 397.93049000000002] 			 [0 1 1 1 2]
[3.1746, 10.715, 386.11040000000003] 			 [0 1 2]
[0.76147, 5.698, 14.557, 378.98352999999997] 			 [0 0 1 2]
[0.34164, 10.458, 389.20035999999999] 			 [0 1 2]
[1.2918, 10.639, 388.06920000000002] 			 [0 1 2]
[0.92699, 19.286, 379.78701000000001] 			 [0 1 2]
[0.8012, 0.38112, 0.62656, 0.26081, 397.93031000000002] 			 [0 0 0 0 2]
[0.87645, 13.439, 385.68455] 			 [0 1 2]
[0.76194, 0.41876, 12.82, 13.507, 372.4923] 			 [0 0 2 2 2]
[0.98922, 10.184, 388.82677999999999] 			 [0 1 2]
[0.40644, 3.6657, 10.243, 385.68486000000001] 			 [0 1 1 2]
[0.95543, 23.526, 375.51857000000001] 			 [0 1 2]
[3.5021, 5.8052, 21.404, 369.28870000000001] 			 [0 1 1 2]
[1.178, 18.33, 380.49200000000002] 			 [0 1 2]
[0.65798, 13.986, 385.35602] 			 [0 1 2]
[2.4423, 18.859, 378.69869999999997] 			 [0 1 2]
[1.6865, 56.16, 342.15350000000001] 			 [0 2 2]
[1.2062, 0.54495, 0.72766, 1.6624, 99.503, 296.35579000000001] 			 [0 0 0 0 2 2]
[2.7911, 12.

In [22]:
#utilization of merge_similar_layers  and make_same_numb_layers functions

mod_layers=[]
mod_layer_lbls=[]
# 
for lyr,lbl in zip(master_l_thicknesses,lith_labels): # changed from loc_layers,lith_labels
    mlyr,mlbl=merge_similar_layers(lyr,lbl)
    mod_layers.append(mlyr)
    mod_layer_lbls.append(mlbl)

for mt,ll in zip(mod_layers, mod_layer_lbls ):
    print(mt,'\t\t\t',ll)

[0.80102, 1.2684900000000001, 397.93049000000002] 			 [0, 1, 2]
[3.1746, 10.715, 386.11040000000003] 			 [0, 1, 2]
[6.4594700000000005, 14.557, 378.98352999999997] 			 [0, 1, 2]
[0.34164, 10.458, 389.20035999999999] 			 [0, 1, 2]
[1.2918, 10.639, 388.06920000000002] 			 [0, 1, 2]
[0.92699, 19.286, 379.78701000000001] 			 [0, 1, 2]
[2.06969, 397.93031000000002] 			 [0, 2]
[0.87645, 13.439, 385.68455] 			 [0, 1, 2]
[1.1806999999999999, 398.8193] 			 [0, 2]
[0.98922, 10.184, 388.82677999999999] 			 [0, 1, 2]
[0.40644, 13.9087, 385.68486000000001] 			 [0, 1, 2]
[0.95543, 23.526, 375.51857000000001] 			 [0, 1, 2]
[3.5021, 27.2092, 369.28870000000001] 			 [0, 1, 2]
[1.178, 18.33, 380.49200000000002] 			 [0, 1, 2]
[0.65798, 13.986, 385.35602] 			 [0, 1, 2]
[2.4423, 18.859, 378.69869999999997] 			 [0, 1, 2]
[1.6865, 398.31349999999998] 			 [0, 2]
[4.14121, 395.85879] 			 [0, 2]
[15.1061, 5.1177, 3.419, 376.35719999999998] 			 [0, 1, 2, 20]
[5.9544500000000005, 2.1745, 391.87105000000003] 			 [

In [23]:
# final_layers,final_labels=segregate_layers(mod_layers,mod_layer_lbls)
# final_layers,final_labels=make_same_numb_layers(mod_layers,mod_layer_lbls)


#create zero thickness layers for missing layers in each location
mod_layers,mod_layer_lbls
def get_unique_labels(mod_layer_lbls):
    res=[]
    for m in mod_layer_lbls:
        res +=m
    unique_lbls=np.unique(res)
    return unique_lbls

unique_lbls=get_unique_labels(mod_layer_lbls)

final_thicks=[]
for m_llyrs,m_llbls in zip(mod_layers,mod_layer_lbls):
    final_thicks.append(get_sortedlayers4loc(m_llyrs,m_llbls,unique_lbls))
unique_lbls,final_thicks

(array([ 0,  1,  2, 20], dtype=int64),
 [[0.80102, 1.2684900000000001, 397.93049000000002, 0],
  [3.1746, 10.715, 386.11040000000003, 0],
  [6.4594700000000005, 14.557, 378.98352999999997, 0],
  [0.34164, 10.458, 389.20035999999999, 0],
  [1.2918, 10.639, 388.06920000000002, 0],
  [0.92699, 19.286, 379.78701000000001, 0],
  [2.06969, 0, 397.93031000000002, 0],
  [0.87645, 13.439, 385.68455, 0],
  [1.1806999999999999, 0, 398.8193, 0],
  [0.98922, 10.184, 388.82677999999999, 0],
  [0.40644, 13.9087, 385.68486000000001, 0],
  [0.95543, 23.526, 375.51857000000001, 0],
  [3.5021, 27.2092, 369.28870000000001, 0],
  [1.178, 18.33, 380.49200000000002, 0],
  [0.65798, 13.986, 385.35602, 0],
  [2.4423, 18.859, 378.69869999999997, 0],
  [1.6865, 0, 398.31349999999998, 0],
  [4.14121, 0, 395.85879, 0],
  [15.1061, 5.1177, 3.419, 376.35719999999998],
  [5.9544500000000005, 2.1745, 391.87105000000003, 0],
  [1.4098, 5.3151, 393.27510000000001, 0],
  [1.5412, 12.654, 385.8048, 0],
  [3.57, 0, 396.429

In [24]:
#final depths with default 0 to 5 or 0-10 layers
final_depths=[]
for ft,el in zip(final_thicks,elevations):
    fd = get_depths(ft,el)
    final_depths.append(fd)
final_depths=np.array(final_depths)
final_thicks=np.array(final_thicks)
# final_depths, final_thicks,unique_lbls      

for ll, lt,fd in zip(lith_labels,liths,final_depths):
    print(ll)
    print(lt)
    print(fd)

[0 1 1 1 2]
['Dry soil with kankar' 'Clay with kankar' 'Clay with kankar'
 'Clay with kankar' 'Hard massive granite']
[-173.    -172.199 -170.93   227.     227.   ]
[0 1 2]
['Soil with Kankar' 'Clay with quartz and weathered pink granite'
 'Pink granite slightly black&white  fine to meadium grains-May be under Saturation']
[-169.    -165.825 -155.11   231.     231.   ]
[0 0 1 2]
['Yellowish sandy clay' 'Yellowish sandy clay' 'Weathered pink granite'
 'Weathered pink Granite to pink granite mafic mineral increase']
[-153.    -146.541 -131.984  247.     247.   ]
[0 1 2]
['Yellowish sandy clay' 'quartz and weathered pink granite Clay'
 'Weathered pink Granite to pink granite mafic mineral increase']
[-155.    -154.658 -144.2    245.     245.   ]
[0 1 2]
['Soil with Kankar'
 'Clay and Weathered pink granite may be under saturation'
 'Weathered pink Granite to pink granite mafic mineral increase']
[-167.    -165.708 -155.069  233.     233.   ]
[0 1 2]
['Dry soil with kankar' 'Weathered grey

In [25]:
# np.append(np.array(N),final_depths,axis=1)
final_depths.shape
N.shape=(N.shape[0],1)
E.shape=(E.shape[0],1)
final_depths.shape,N.shape
xy_final_depths=np.append(np.array(E),np.append(np.array(N),final_depths,axis=1),axis=1)
np.save(base_folder+'final_depths_labes_summary.npy',(xy_final_depths,unique_lbls,lith_dict_summay))

In [14]:
unique_lbls

array([ 0,  1,  2,  3, 30], dtype=int64)